# Accessing data
## Access raw data from Azure container
- Create connection to Azure Blob Storage
- Convert the CSV file into parquet file format
- Upload the parquet file into Bronze layer/folder

## Access uploaded parquet file from the Bronze layer
- Download the specific blob
- Load data into dataframes using Pandas


### Installation of Azure Storage Blob 

In [1]:
# pip install azure-storage-blob

### Create a connection to the Azure Blob Storage

In [2]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [3]:
# Connection string  and Container Name containing Azure Blob Storage account information
connection_string = "DefaultEndpointsProtocol=https;AccountName=**your_account_name**;AccountKey=**your_account_key**==;EndpointSuffix=core.windows.net"
container_name = "training"  ## your container name in Azure

In [4]:
# Create a BlobServiceClient object to interact with Azure Blob Storage
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

In [5]:
# Connect to Azure Blob Storage and retrieve a ContainerClient for a specific container
container_client = blob_service_client.get_container_client(container_name)

In [6]:
# List blobs in the container
blob_list = container_client.list_blobs()
for blob in blob_list:
    print(blob.name) 

01-Raw
01-Raw/dqdatademo.csv
02-Bronze
02-Bronze/dqdatademo.parquet
03-Silver
04-Gold


### Converting the .csv to .parquet

In [7]:
 blob_name = "01-Raw/dqdatademo.csv"  # Specify the name of the CSV file

In [8]:
 blob_client = container_client.get_blob_client(blob_name)

In [9]:
# Download CSV data into memory
csv_data = blob_client.download_blob().readall().decode("utf-8")

In [10]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from io import StringIO

In [11]:
# Convert CSV data to pandas DataFrame
df = pd.read_csv(StringIO(csv_data))

In [12]:
# Convert DataFrame to Parquet format
parquet_file_path = "dqdatademo.parquet"
table = pa.Table.from_pandas(df)
pq.write_table(table, parquet_file_path)

### Upload the .parquet into Bronze file

In [13]:
# Upload the Parquet file back to Azure Blob Storage
parquet_blob_client = container_client.get_blob_client("02-Bronze/dqdatademo.parquet")
with open(parquet_file_path, "rb") as data:
    parquet_blob_client.upload_blob(data)

## Create connection to Azure to access the uploaded parquet file in Bronze folder

## Load the parquet file

In [14]:
import io
# Download a specific blob (e.g., 'sample_data.parquet') as a stream and load it into a pandas DataFrame
blob_name = '02-Bronze/dqdatademo.parquet'  # Replace with your blob name
blob_client = container_client.get_blob_client(blob_name)

# Download the blob content
blob_data = blob_client.download_blob().readall()

# Load the data into a pandas DataFrame using pyarrow
data = pq.read_table(io.BytesIO(blob_data)).to_pandas()

# Display the DataFrame
print(data.head())

        id              name                         email    country  age
0   502797    James Williams            dreyes@example.com    Germany   24
1  3955827      Eileen Perez       vancesierra@example.com        USA   46
2  2985040  Brittney Daniels        brendaowen@example.net    Germany   47
3  1678851      Linda Wilson  contrerasrebecca@example.net  Australia   19
4  4307350       Jacob Tapia            gallen@example.com    Germany    8


In [15]:
# Convert Arrow Table to pandas DataFrame
df = table.to_pandas()

In [16]:
# Display the original DataFrame
print("Original DataFrame:")
df.sample(5)

Original DataFrame:


,id,name,email,country,age
2722431,2112089,Sarah Franklin,sethwilson@example.net,Singapore,41
499280,1525501,Jessica Morse,qpeterson@example.org,Australia,25
4199703,3125303,Stephen Maldonado,travis76@example.com,China,79
2496385,376548,Amber Schmidt,xjohnson@example.org,Malaysia,20
3842164,749767,Laurie Anderson,jbaker@example.net,Singapore,92


## Split name column to further breakdown and prepare the dataset
- divide into 7 column; id, Prefix, name, Title, age, email, country
- Drop 'prefix' column as the 'Mr or Ms' title unable to be referenced to other columns, however, title "Dr." is deemed significant
- The pattern [IVX] is a character set or character class in regular expressions. It matches a single character that is either "I", "V", or "X". Here's what each character in the set 

In [17]:
import re
import pandas as pd


def separate_name(name):
    # Define regex pattern
    pattern = r'(?P<prefix>Mr\.|Ms\.|Mrs\.|Miss|Dr\.)?\s*(?P<name>.+?)(?:\s+(?P<title>[IVX]+|PhD|MD|DVM|DDS|Jr.))?$'

    match = re.match(pattern, name)
    if match:
        title = match.group('prefix')
        abbrev = match.group('title')
        name = match.group('name').strip()
        
        return title, name, abbrev
    else:
        return None, None, None
        
# Assign new variable to splitted name column parquet dataset
df_splitted_name = pd.DataFrame(df) # <-- dqdatademo dataset

# Apply the function to each row
df_splitted_name[['prefix', 'name', 'title']] = df_splitted_name['name'].apply(separate_name).apply(pd.Series)

In [18]:
df_splitted_name.sample(10)

,id,name,email,country,age,prefix,title
216476,1356659,Matthew Huerta,joshua75@example.com,Malaysia,81,None,None
2137940,20466,Frederick Davis,jasminknight@example.com,Canada,13,None,None
1918822,2254486,Aaron Harris,abailey@example.org,Australia,27,None,None
4031388,297752,Courtney Taylor,sburke@example.org,France,8,None,None
3499828,2351961,Andrea Curtis,jerrymoore@example.net,India,81,None,None
4746294,3848625,Matthew Espinoza,cindy82@example.org,China,50,None,None
4908700,2496901,Eric Huffman,rowekarla@example.net,Malaysia,22,None,None
945081,3025297,Patricia Williams,floreschristina@example.com,Australia,86,None,None
4844664,3780060,Lauren Wright,courtney08@example.net,UK,64,None,None
1152944,3491809,Angela Johnson,efernandez@example.net,Germany,35,None,None


In [19]:
df_splitted_name.shape

(5000000, 7)

### Rearrange columns for readability

In [20]:
# Specify the desired column order
desired_columns = ['id','prefix', 'name', 'title', 'email', 'country', 'age']


# Reassign the DataFrame with the columns in the desired order
new_df = df_splitted_name[desired_columns]

In [21]:
new_df.sample(10)

,id,prefix,name,title,email,country,age
2607510,1660713,None,Tracy Singh,None,zgibson@example.com,Nigeria,44
3925422,4234425,None,Denise Mitchell,None,hamiltonlinda@example.net,India,34
250111,1152739,None,Robert Griffin,None,desiree32@example.com,New Zealand,88
668072,3836462,None,Jeremy Thomas,None,kevin94@example.net,Malaysia,33
3460207,136622,None,Justin Hernandez,None,charleslucas@example.org,Canada,53
1067325,2600090,None,Jeffrey Love,None,jonathancooper@example.org,Malaysia,37
3358981,3527423,None,Gwendolyn Smith,None,christopherscott@example.com,USA,55
639103,3065682,None,Glen Cook,None,christina39@example.org,India,96
2197494,1243472,None,Tiffany Moore,None,imejia@example.com,Malaysia,84
404113,4284898,None,Miranda Armstrong,None,frank24@example.com,Malaysia,63


### Before drop the prefix column, notice there are meaningful value; "Dr."

In [22]:
# Check if 'title' column contains 'Jr.'
search_string = 'Dr'
mask = new_df['prefix'].astype(str).str.contains(search_string, case=False)

# Filter DataFrame based on the mask
result = new_df[mask]

print(result)


              id prefix               name title                        email  \
168      3107966    Dr.     Rachel Webster  None       johnspears@example.com   
270      3592176    Dr.    Reginald Waller  None  vanessasandoval@example.net   
410       617151    Dr.        Shirley Lee  None            mtodd@example.com   
544      4359633    Dr.      Ricardo Brown  None   francojennifer@example.com   
654      1439545    Dr.   Yolanda Marshall    MD      stephanie98@example.net   
...          ...    ...                ...   ...                          ...   
4999560  3062506    Dr.       Cindy Harris  None       marshpedro@example.net   
4999660  1647225    Dr.  Matthew Hernandez  None    michaelturner@example.com   
4999692  3022538    Dr.    Kimberly Parker  None         lauren15@example.net   
4999753  2884411    Dr.        Guy Vaughan   PhD           qcohen@example.com   
4999920  2696234    Dr.        Bryan Woods    MD  stanleyschwartz@example.com   

             country  age  

#### 29528 rows consist of Dr. prefixes. 


# Start Data Profiling Process
Measure Data Quality on Data that covering the following aspects:
1. Completeness - Replace meaningful value, "Dr." with mode value in title column
2. Consistency - Format consistent on email column - Alphabetical value
3. Consistency - Email domain column
4. Accuracy - Accuracy on the relation of name and email columns
5. Validity - Validity for title, age and email columns
6. Uniqueness - Deduplication on id and name columns


# 1. Completeness - prefix column with title column

- Since "Dr." is meaningful data, we decided to replace it with mode value in the title column. 
- So, instead of "Dr. Christian Taylor", it will be "Christian Taylor < mode value>"

Accepted data:
- Prefix other than "Dr." 
- Title not null

Rejected Data:
- prefix column has "Dr."
- title column is null

#### Dividing Accepted dataset and Rejected Dataset

In [23]:
# Check existence of "Dr." in 'prefix' column
dr_prefix_mask = new_df['prefix'].str.contains(r'^Dr\.', regex=True, na=False)

# Check for null values in 'title' column
null_title_mask = new_df['title'].isnull()

# Filter DataFrame based on conditions

#Rejected dataset
rejected_df = new_df[dr_prefix_mask & null_title_mask]

#Accepted dataset
accepted_df = new_df[~(dr_prefix_mask & null_title_mask)]

In [24]:
print("Accepted DataFrame:")
accepted_df.sample(20)

Accepted DataFrame:


,id,prefix,name,title,email,country,age
3892264,1244760,None,Mary Lopez,None,barrybailey@example.com,Malaysia,11
2252309,4368370,None,Nicholas Carroll,None,lwilliams@example.org,New Zealand,57
669652,589089,None,Charles Davis,None,lmiller@example.net,Singapore,18
2480028,66780,None,Victor Jackson,None,dbecker@example.net,UK,25
2265893,1387267,None,Michelle Park,None,lisa95@example.com,France,25
4778791,24953,None,Kayla Oneill,None,savannahmolina@example.net,Malaysia,93
135621,587611,None,Jennifer Stafford,None,browngary@example.org,Malaysia,1
1781527,190364,None,Michael Lang,None,ewilliams@example.org,Germany,70
1900105,2351783,None,Mary Duran,None,simpsondaniel@example.com,USA,32
1184351,4458648,None,Cynthia Barr,None,samuel64@example.net,New Zealand,27


In [25]:
rejected_df.sample(10)

,id,prefix,name,title,email,country,age
2178113,3317671,Dr.,Melanie Freeman,None,tracy07@example.net,Australia,98
4691943,1395530,Dr.,Stephanie Miller,None,stevenschristopher@example.org,Australia,95
1929851,963148,Dr.,Wendy Crawford,None,shaunwright@example.org,France,9
3041369,1444922,Dr.,Christine Miller,None,ksmith@example.com,New Zealand,55
1554957,2713375,Dr.,Joseph Townsend,None,umatthews@example.org,Canada,4
1485934,2131861,Dr.,Brent Keith,None,jchambers@example.org,UK,68
4473489,2332987,Dr.,Hannah Christian,None,walkeralejandra@example.org,Malaysia,80
1780247,1865576,Dr.,Sharon Jones,None,jennifer47@example.org,Canada,42
4141471,421744,Dr.,Amanda Ferguson,None,nathaniel16@example.com,China,62
816121,2438442,Dr.,Shawn Smith,None,craig94@example.com,Australia,15


In [26]:
rejected_df.shape

(22151, 7)

In [27]:
accepted_df.shape

(4977849, 7)

In [28]:
rejected_df_copy = rejected_df.copy()

### Modify the rejected dataset
- Replace the "Dr." inside the prefix column with **mode value** inside the title column


#### Mode value in title column

In [29]:
# Calculate mode value of 'title' column
mode_title = new_df['title'].mode()[0] if not new_df['title'].isnull().all() else None
print(mode_title)

MD


In [30]:
# Replace null values in 'title' column with mode value
if mode_title is not None:
    rejected_df.loc[:, 'title'] = mode_title

rejected_df.sample(5)

,id,prefix,name,title,email,country,age
4327088,194798,Dr.,Tracey Russo,MD,fsmith@example.com,UK,49
1187866,4249191,Dr.,Alicia Wilson,MD,amanda54@example.net,Canada,78
1758812,907359,Dr.,Matthew Morse,MD,tmontoya@example.org,Singapore,23
4090144,2850634,Dr.,Crystal Chambers,MD,wayneware@example.net,New Zealand,59
4142282,2749182,Dr.,Heather Nielsen,MD,mitchell55@example.com,UK,23


#### Merge the modified rejected dataset with accepted dataset

In [31]:
import pandas as pd
## Code snippet to merge data ~ .. 'outer' is like full join
dqCompletenessTitle = pd.merge(accepted_df,rejected_df,how='outer')

In [32]:
dqCompletenessTitle.sample(10)

,id,prefix,name,title,email,country,age
669908,602649,None,Charles Hawkins,None,youngrebecca@example.net,China,22
504083,453547,None,Michelle Smith,None,marissa21@example.com,China,68
3088923,2779447,None,Caitlin Tran,None,michaelgreen@example.org,France,27
217187,195452,None,Stephanie Rojas,None,katierandolph@example.org,India,75
1298372,1168148,None,Norman Nguyen,None,justinquinn@example.org,Singapore,36
3685473,3316334,None,Casey Brown,None,meghanpotter@example.com,China,31
2974911,2676852,None,Melissa Johnson,None,patrickfisher@example.org,Malaysia,5
899080,808889,None,Ricky Lee,None,ryananderson@example.com,New Zealand,49
2442985,2197874,None,Jeffrey Frey,None,adamosborne@example.com,USA,31
991469,891942,None,Heather Rogers,None,tmiller@example.com,Nigeria,19


#### Check one of the value in rejected dataset;  "Dr. Rachel Webster MD" instead of "Dr. Rachel Webster"

In [33]:
# Check if 'title' column contains 'Jr.'
search_string = '3107966'
mask = dqCompletenessTitle['id'].astype(str).str.contains(search_string, case=False)

# Filter DataFrame based on the mask
result = dqCompletenessTitle[mask]

print(result)


              id prefix            name title                   email  \
3454063  3107966    Dr.  Rachel Webster    MD  johnspears@example.com   
3454064  3107966    Dr.  Rachel Webster    MD  johnspears@example.com   

          country  age  
3454063  Malaysia   41  
3454064  Malaysia   41  


#### Now, drop prefix column

In [34]:
# Drop the 'Prefix' column
df_modified = dqCompletenessTitle.drop('prefix', axis =1)

In [35]:
df_modified.sample(10)

,id,name,title,email,country,age
3143725,2828849,Anthony Bates,None,jill65@example.net,France,5
3006159,2704913,Holly Wang,None,lrice@example.org,UK,38
649658,584446,Lisa Chavez,None,rebecca11@example.org,China,41
1777559,1599268,George Schmitt,None,tinalewis@example.org,USA,73
2735972,2461815,Jake Pena,None,tiffanyrhodes@example.net,Germany,41
187468,168722,Jennifer Taylor,None,mitchelljoseph@example.net,China,100
4854568,4369039,Russell Patel,None,nathaniel97@example.com,Canada,14
526328,473532,Kenneth Hall,None,rachelhouston@example.org,USA,90
528866,475808,William Martin,None,poncechristopher@example.org,Australia,44
578022,519925,Nicole Barr,None,amber92@example.org,India,12


In [36]:
df_modified.shape

(5000000, 6)

# 2. Consistency for 'email' column
1. Split 'email' column into 'email_ID' and 'domain'
2. Filter 'email_ID' into accepted and rejected (to ensure all 'email_id' is *alphabetic* only)
3. Modify rejected 'email_ID' value (alphanumeric -> alphabet ONLY ; eg: joshua91 -> joshua)
4. Merge modified rejected with accepted = total rows ; **Consistency of email column is achieved**


- Continue from **Completeness** variable
- Make a copy
- Check shape before and after copy

In [37]:
df_modified.shape # use variable from Completeness part

(5000000, 6)

In [38]:
df_new = df_modified.copy()

In [39]:
df_new.shape

(5000000, 6)

#### Split email into 3 columns ('email_id', 'domain', 'TLD')

In [40]:
# identify and split email into 2 columns
import re

# Function to separate email into username, domain with @, and TLD
def separate_email_df(email):
    email_id, domain = email.split('@')
    domain_name, tld = domain.rsplit('.', 1)
    return pd.Series([email_id, "@" + domain_name,"." + tld], index=['email_id', 'domain', 'TLD'])

# usage with DataFrame:  ** df_new is the variable
df_new[['email_id', 'domain', 'TLD']] = df_new['email'].apply(separate_email_df).apply(pd.Series)

In [41]:
df_new.sample(10)

,id,name,title,email,country,age,email_id,domain,TLD
3757454,3381101,Michelle Sharp,None,allenbrenda@example.com,France,70,allenbrenda,@example,.com
4529261,4076173,Cassandra Lewis,None,dayphilip@example.org,UK,80,dayphilip,@example,.org
449084,404056,Paula Case,None,alexanderphillips@example.com,Germany,83,alexanderphillips,@example,.com
2163477,1946424,David Browning,None,gjohnson@example.com,France,6,gjohnson,@example,.com
2220393,1997581,Alexander Burke,MD,cheryl91@example.net,Malaysia,76,cheryl91,@example,.net
4717126,4245493,Michelle Jenkins,None,hparks@example.net,Malaysia,53,hparks,@example,.net
2749295,2473832,Charles Herring,None,mperez@example.org,Australia,3,mperez,@example,.org
4364200,3927504,Teresa Cohen,None,jkane@example.org,Malaysia,79,jkane,@example,.org
891674,802248,Colleen Hall,None,timothyhiggins@example.org,India,42,timothyhiggins,@example,.org
181330,163218,Joyce Robinson,None,dan05@example.net,Singapore,32,dan05,@example,.net


In [42]:
df_new.shape

(5000000, 9)

In [43]:
# copy df_new for backup
df_consistency = df_new.copy()

In [44]:
df_consistency.shape

(5000000, 9)

In [45]:
df_consistency.sample(5)

,id,name,title,email,country,age,email_id,domain,TLD
3173428,2855536,Joseph Ayala,None,mrichardson@example.org,Malaysia,74,mrichardson,@example,.org
4802935,4322598,Phillip Cummings,None,meganjackson@example.com,Malaysia,48,meganjackson,@example,.com
4189357,3770072,Matthew Davis,None,tfrancis@example.org,Canada,32,tfrancis,@example,.org
4092031,3682583,Timothy Jackson,None,kimberly34@example.net,Malaysia,67,kimberly34,@example,.net
1483075,1334288,Clifford Downs,None,karenbray@example.net,Germany,98,karenbray,@example,.net


- Drop 'email' because can be referred from 'email_id', 'domain' and 'TLD'

In [46]:
df_consistency = df_consistency.drop('email', axis=1)

In [47]:
df_consistency.sample(3)

,id,name,title,country,age,email_id,domain,TLD
2602768,2341881,Stanley Banks,None,UK,15,amykim,@example,.org
2427282,2183719,Teresa Monroe,None,Germany,46,fguzman,@example,.com
2987075,2687784,John Holland,None,China,93,billyli,@example,.com


- Rearrange for readability

In [48]:
# rearrange columns into 'id', 'name','title', 'email_id', 'domain', 'country', 'age'
rearrange_col = ['id', 'name','title', 'email_id', 'domain', 'TLD', 'country', 'age']

# Reassign to new_df_2 with the desired column
df_consistency = df_consistency[rearrange_col]

In [49]:
df_consistency.sample(10)

,id,name,title,email_id,domain,TLD,country,age
686333,617433,Sarah Cline,None,kingbrian,@example,.org,Singapore,58
4917117,4425381,Gregory Williams,None,sandovalelizabeth,@example,.com,Malaysia,48
51814,46657,Chris Johnson,None,medinamelissa,@example,.com,France,38
3016854,2714520,Dustin Davies,None,lijennifer,@example,.com,China,56
511863,460529,Thomas Morris,None,paige55,@example,.net,India,37
1199404,1079029,Kimberly Williams,None,dmarquez,@example,.net,Malaysia,60
1170521,1052932,Brian Page,None,robert56,@example,.com,New Zealand,19
2267512,2039956,Lisa Peterson,None,klawrence,@example,.net,China,83
4849040,4364059,Matthew Lowe,None,michaelrice,@example,.com,France,73
1626103,1462967,Michael Torres,None,douglas56,@example,.com,India,3


#### Working on *consistency* of 'email_id' column
- accepted = combination of Fname + Lname lowercase
- rejected = contains **numerical value**
* seperate into accepted , rejected email_id

In [50]:
# source : df_consistency ; seperate accepted, rejected

# Function to check if email_id is alphabet only
def is_alphabetic(email):
    return email.replace('@', '').isalpha()

# Apply the function and split the DataFrame
accepted_consistency = df_consistency[df_consistency['email_id'].apply(is_alphabetic)]  ## accepted
rejected_consistency = df_consistency[~df_consistency['email_id'].apply(is_alphabetic)] ## rejected

In [51]:
accepted_consistency.shape

(3750360, 8)

In [52]:
rejected_consistency.shape

(1249640, 8)

In [53]:
# sample accepted
accepted_consistency.sample(5)

,id,name,title,email_id,domain,TLD,country,age
2662486,2395607,Rebecca Thomas,None,davidmcdaniel,@example,.net,Canada,51
525875,473125,Emily Martinez,None,mendezleslie,@example,.org,Australia,10
4552835,4097400,Michelle Rodriguez,None,monicastewart,@example,.com,China,14
3489453,3139860,Lisa West,None,rodriguezamy,@example,.org,China,53
4443709,3999051,Alexander Pearson,None,kjackson,@example,.net,China,55


In [54]:
rejected_consistency.sample(5)

,id,name,title,email_id,domain,TLD,country,age
3796443,3416204,Christina Mcdaniel,None,madison21,@example,.com,Malaysia,30
3971989,3574343,Jack Nelson,None,lucas87,@example,.org,UK,11
2037689,1833219,Steven Bradley,None,daniel23,@example,.com,India,15
3872166,3484452,Jeremiah Lucas,None,shelby06,@example,.net,UK,14
1177947,1059638,Christopher Brown,None,lisa78,@example,.com,Australia,33


- accepted_consistency + rejected_consistency = total of rows before splitting (5 Million)

#### Modify rejected dataframe:
discard numerical value from the 'email_id' string, and keep alphabetical string ONLY ; example-> Joshua91 ==> Joshua

In [55]:
# Modify values in 'email_id'
# Define a function to modify the username
def modify_email(email_id):
    # Check if the username contains numerical characters
    if any(char.isdigit() for char in email_id):
        # Remove numerical characters
        modified_email = ''.join(filter(lambda x: not x.isdigit(), email_id))
        return modified_email
    else:
        return email_id

# Apply the function to the 'username' column
rejected_consistency['email_id'] = rejected_consistency['email_id'].apply(modify_email)

C:\Users\user\AppData\Local\Temp\ipykernel_11952\897668353.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rejected_consistency['email_id'] = rejected_consistency['email_id'].apply(modify_email)


In [56]:
rejected_consistency.sample(10)

,id,name,title,email_id,domain,TLD,country,age
3378832,3040412,Katherine Gonzalez,None,gregory,@example,.org,Germany,61
1842972,1658031,Loretta Steele,None,danielle,@example,.com,India,16
3028977,2725438,Cody Parker,None,jacob,@example,.com,UK,4
1352693,1217094,Pamela Snyder,None,jerry,@example,.org,New Zealand,12
206370,185746,Erik Garcia,None,roger,@example,.com,Germany,54
735193,661412,Wesley Whitney,None,stephanie,@example,.org,USA,99
4536285,4082517,Robert Kane,None,veronica,@example,.com,Malaysia,7
1014723,912798,Joshua Nunez,None,victoria,@example,.com,Australia,37
2611406,2349695,Jennifer Gonzalez,None,crystal,@example,.org,UK,32
2968690,2671250,Jacqueline Figueroa,PhD,jose,@example,.net,Malaysia,1


#### Merge accepted and rejected dataset
 *new merged_df called -> merged_consistency

In [57]:
## Code snippet to merge data ~ ..
merged_consistency = pd.merge(accepted_consistency, rejected_consistency, how='outer')

In [58]:
merged_consistency.shape

(5000000, 8)

In [59]:
# copy merged_consistency ### for presentation screenshot
merged_consistency_1 = merged_consistency.copy()

In [60]:
# merge email id and domain with new variable => new_email
merged_consistency_1['new_email'] = merged_consistency_1['email_id'] + merged_consistency_1['domain']+ merged_consistency_1['TLD']

In [61]:
# rearrange columns into 'id', 'name','title', 'email_id', 'domain', 'country', 'age'
rearrange_col_mc = ['id', 'name','title', 'new_email','country', 'age']

# Reassign to new_df_2 with the desired column
merged_consistency_1 = merged_consistency_1[rearrange_col_mc]

In [62]:
print(merged_consistency_1[merged_consistency_1['id'].isin(range(1, 9))].sort_values(by='id', ascending=True))

   id              name title                  new_email      country  age
0   1     Gloria Montes  None  amandaspencer@example.org     Malaysia   68
1   2  Ricardo Anderson   PhD         justin@example.com  New Zealand   11
2   3        Sara Lewis  None      smithjose@example.net      Germany   21
3   4    Troy Maldonado  None  allisonmurphy@example.com    Australia   68
4   5   Cheyenne Baxter  None      jaredbell@example.org        China   20
5   6       David Smith  None         jriley@example.org     Malaysia   22
6   6       David Smith  None         jriley@example.org     Malaysia   22
7   7       David Lyons  None  freemanjeanne@example.com           UK   78
8   8     Tommy Johnson  None          kelly@example.org           UK   31


In [63]:
merged_consistency.sample(3)

,id,name,title,email_id,domain,TLD,country,age
3061930,2755119,Mitchell Avila,None,melissa,@example,.net,Germany,81
2705916,2434746,Lisa Rogers,None,boltonrebecca,@example,.org,Malaysia,91
3452448,3106501,David Johnston,None,kday,@example,.com,Nigeria,5


# 2.2 Consistency for 'email_domain'
- convert all top level domain (TLD) '.net','.org','.com'==> .com   **Commercial use**

1. Split 'email' column into 'email_no_extension' and 'email_extension'  **extension == TLD**
2. Apply the extension constraint/condition
3. Merge 'email_without_extension' with 'new_extension' as 'modified_email'
4. Replace value of column 'email' with 'modified_email' ; and drop the 'modified_email' column

In [64]:
df_extension = merged_consistency.copy()

In [65]:
df_extension.shape

(5000000, 8)

In [66]:
df_extension.sample(3)

,id,name,title,email_id,domain,TLD,country,age
3616810,3254459,Judy Smith,None,dana,@example,.com,USA,91
267371,240566,Steven Castro,None,lindarussell,@example,.org,New Zealand,43
1156636,1040379,Catherine Riggs,None,bryan,@example,.org,France,39


In [67]:
# Count the occurrences of each extension in 'modify_extension'
extension_counts = df_extension['TLD'].apply(lambda x: x.split('.')[-1]).value_counts()
print(extension_counts)

TLD
net    1668773
com    1665914
org    1665313
Name: count, dtype: int64


### Provide new extension for "email_without_extension" :
- Change all TLD into ==> ".com"   **Commercial use** 

In [68]:
def modify_extension(row):
    new_extension = '.com'
    return new_extension

# Create the new 'modify_extension' column
df_extension['new_TLD'] = df_extension.apply(modify_extension, axis=1)

In [69]:
df_extension.sample(5)

,id,name,title,email_id,domain,TLD,country,age,new_TLD
2030223,1826536,Meghan Chase,None,ajones,@example,.com,Malaysia,37,.com
4103181,3692625,Carrie Black,None,ugould,@example,.org,Malaysia,65,.com
3841281,3456603,Jennifer Ruiz,None,solomonchristopher,@example,.org,France,31,.com
2047013,1841651,Eddie Sanchez,None,skidd,@example,.net,UK,35,.com
1406892,1265778,Amy Lopez,None,xbell,@example,.net,New Zealand,27,.com


In [70]:
# Count the occurrences of each extension in 'modify_extension'
extension_counts_new = df_extension['new_TLD'].apply(lambda x: x.split('.')[-1]).value_counts()
print(extension_counts_new)

new_TLD
com    5000000
Name: count, dtype: int64


In [71]:
# Assign value of column 'TLD' with column 'new_TLD'
df_extension['TLD'] = df_extension['new_TLD']

In [72]:
df_extension.sample(3)

,id,name,title,email_id,domain,TLD,country,age,new_TLD
4607904,4147037,Dennis Gardner,None,bchung,@example,.com,Australia,34,.com
280904,252743,Carla Allen,None,adamcook,@example,.com,Australia,61,.com
3875674,3487597,Jeffery Morgan,None,davidthomas,@example,.com,Nigeria,23,.com


In [73]:
# Drop columns
df_extension = df_extension.drop(['new_TLD'], axis=1)

In [74]:
df_extension.sample(3)

,id,name,title,email_id,domain,TLD,country,age
2389832,2150089,Kenneth Carter,None,lturner,@example,.com,Malaysia,99
1638741,1474389,Dawn Wallace,None,dvazquez,@example,.com,Singapore,68
2275601,2047242,Kimberly Zhang,None,maryscott,@example,.com,UK,58


In [75]:
df_extension_example = df_extension.copy()

In [76]:
# merge all (email_id, domain, TLD) as ==> 'email'
df_extension_example['email'] = df_extension_example['email_id'] + df_extension_example['domain'] + df_extension_example['TLD']

In [77]:
#rearrange column
desirecol = ['id','name','title','email','country','age']

df_extension_example = df_extension_example[desirecol]

In [78]:
print(df_extension_example[df_extension_example['id'].isin(range(1, 9))].sort_values(by='id', ascending=True))

   id              name title                      email      country  age
0   1     Gloria Montes  None  amandaspencer@example.com     Malaysia   68
1   2  Ricardo Anderson   PhD         justin@example.com  New Zealand   11
2   3        Sara Lewis  None      smithjose@example.com      Germany   21
3   4    Troy Maldonado  None  allisonmurphy@example.com    Australia   68
4   5   Cheyenne Baxter  None      jaredbell@example.com        China   20
5   6       David Smith  None         jriley@example.com     Malaysia   22
6   6       David Smith  None         jriley@example.com     Malaysia   22
7   7       David Lyons  None  freemanjeanne@example.com           UK   78
8   8     Tommy Johnson  None          kelly@example.com           UK   31


- Consistency of email username and email TLD is achieved ('alphabetic', '.com')
- continue to Accuracy part using variable ==> 'df_extension'

# 3. Accuracy for relation between column 'name' and 'email'
- Print out sample, ensure its correctness (rows & columns) before proceeds
1. Generate new column "expected_email_id" , based off 'name' column with constraints all lowercase and no whitespace
2. Check assumption that 'email_ID' == 'expected_email_id'  ; 19 rows matched with assumption (before deduplication process)
3. Seperate/Split 'email_ID' into accepted and rejected
4. Modify rejected 'email_ID' by replacing/re-assign value of "expected_email_id" into "email_ID"
5. Merge 'modified rejected' and 'accepted' as/for column 'email_ID' ; step 9
6. Drop 'expected_email_id' and 'matched' columns  (Step 10)
7. Merge 'email_ID' and 'domain' as 'email' column  **Accuracy between name & email is achieved** 

#### Continue from Consistency part

In [79]:
def clean_name_df(name):
    # Convert the name to lowercase and remove whitespace
    cleaned_name = name.lower().replace(" ", "")
    return cleaned_name

df_extension['expected_email_id'] = df_extension['name'].apply(clean_name_df)

In [80]:
df_extension.sample(10)

,id,name,title,email_id,domain,TLD,country,age,expected_email_id
3145666,2830591,Katherine Proctor,None,hpham,@example,.com,India,12,katherineproctor
4590434,4131303,Nancy Shannon,None,ksalazar,@example,.com,India,90,nancyshannon
4926708,4434027,John Gould,DVM,adriananderson,@example,.com,Malaysia,64,johngould
3721423,3348698,Robert Blair,None,ruizcatherine,@example,.com,China,81,robertblair
2520832,2268135,Joseph Bradley,None,marissa,@example,.com,India,35,josephbradley
3043534,2738514,Angela Sherman,None,fredmontgomery,@example,.com,Nigeria,61,angelasherman
791995,712510,Laurie Mccall,None,andrew,@example,.com,New Zealand,54,lauriemccall
453491,408010,Alex Beltran,None,kmcintosh,@example,.com,India,17,alexbeltran
356389,320579,Melissa Moore,None,fowlerpatrick,@example,.com,Nigeria,35,melissamoore
3285801,2956609,Casey Black,None,james,@example,.com,Singapore,72,caseyblack


In [81]:
# rearrange columns into 'id','Name','expected_email_id','email_id', 'Domain', 'country', 'age'
rearrange_col_acc = ['id', 'name', 'title', 'expected_email_id', 'email_id', 'domain', 'TLD' , 'country', 'age']

# Reassign to new_df_2 with the desired column
df_accuracy = df_extension[rearrange_col_acc]

In [82]:
df_accuracy.sample(5)

,id,name,title,expected_email_id,email_id,domain,TLD,country,age
1107387,996092,Sandra Neal,None,sandraneal,fitzgeraldmanuel,@example,.com,Malaysia,21
4307124,3876195,Miguel Maxwell,None,miguelmaxwell,megan,@example,.com,Australia,93
4226249,3803315,Mackenzie Williams,None,mackenziewilliams,iodonnell,@example,.com,China,94
1519747,1367309,Melissa Rose,None,melissarose,benjamin,@example,.com,Singapore,96
1756487,1580357,Dawn Banks,None,dawnbanks,michelle,@example,.com,Germany,32


#### Check for match and compare between expected_email_id (name) and email_id column ;
 * if matched = 'True', else false ; hence got the accuracy

In [83]:
# Function to CHECK if email_id matches expected_email_id
def check_match(row):
    return row['expected_email_id'] == row['email_id']

# Add new column to store the value of 'matched'
df_accuracy['matched'] = df_accuracy.apply(check_match, axis=1)

C:\Users\user\AppData\Local\Temp\ipykernel_11952\780265807.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_accuracy['matched'] = df_accuracy.apply(check_match, axis=1)


In [84]:
# Count occurrences of True and False in the 'matched' column
matched_counts = df_accuracy['matched'].value_counts()

# Print the counts
print("True count:", matched_counts.get(True, 0))
print("False count:", matched_counts.get(False, 0))

True count: 19
False count: 4999981


- Expected result should have True count: 19 (contains duplicate)

#### Identify accepted (matched) and rejected (unmatched) email id, and seperate/store it in variable

In [85]:
# Filter the DataFrame to show only rows where 'matched' is True
matched_rows = df_accuracy[df_accuracy['matched']]

In [86]:
matched_rows.sample(19)

,id,name,title,expected_email_id,email_id,domain,TLD,country,age,matched
421436,379156,Cynthia Johnson,None,cynthiajohnson,cynthiajohnson,@example,.com,Australia,37,True
809935,728620,Sara Gonzalez,None,saragonzalez,saragonzalez,@example,.com,India,64,True
1038441,934077,Patrick Warren,None,patrickwarren,patrickwarren,@example,.com,USA,90,True
4050444,3645070,Robert Miller,None,robertmiller,robertmiller,@example,.com,USA,60,True
4251192,3825729,Jennifer Williams,None,jenniferwilliams,jenniferwilliams,@example,.com,New Zealand,4,True
494478,444892,Matthew Johnson,None,matthewjohnson,matthewjohnson,@example,.com,Malaysia,78,True
2465935,2218632,Brandon Wilson,None,brandonwilson,brandonwilson,@example,.com,Malaysia,58,True
2801327,2520629,Angela Williams,None,angelawilliams,angelawilliams,@example,.com,Germany,5,True
1038440,934077,Patrick Warren,None,patrickwarren,patrickwarren,@example,.com,USA,90,True
3648857,3283256,Amanda Phillips,None,amandaphillips,amandaphillips,@example,.com,Germany,1,True


In [87]:
matched_rows.shape

(19, 10)

In [88]:
# Filter the DataFrame to show only rows where 'matched' is False
unmatched_rows = df_accuracy[~df_accuracy['matched']]

# Display the unmatched rows
unmatched_rows.sample(10)

,id,name,title,expected_email_id,email_id,domain,TLD,country,age,matched
615748,553884,Lisa Harris,None,lisaharris,ymoore,@example,.com,China,48,False
4086013,3677160,Gregory Li,None,gregoryli,barnesalbert,@example,.com,Malaysia,39,False
4920110,4428082,Nicholas Schaefer,None,nicholasschaefer,richard,@example,.com,Nigeria,60,False
2628500,2365040,Cheryl Johnson,None,cheryljohnson,douglashaas,@example,.com,Malaysia,93,False
1089697,980230,Nicholas Rogers,None,nicholasrogers,harriswilliam,@example,.com,Germany,23,False
3417699,3075270,Tara Contreras,None,taracontreras,patricialawson,@example,.com,UK,55,False
972958,875239,Lisa Herrera,None,lisaherrera,dawn,@example,.com,Australia,20,False
160163,144139,Brian Taylor,None,briantaylor,qwilkinson,@example,.com,Australia,72,False
3883298,3494452,Jessica Gonzales,None,jessicagonzales,daniel,@example,.com,India,9,False
4556381,4100586,Colleen Costa,None,colleencosta,brooksmegan,@example,.com,Canada,96,False


In [89]:
unmatched_rows.shape

(4999981, 10)

#### Modifying the rejected/unmatched rows to achieve consistency and accuracy of rows

In [90]:
# For consistency and accuracy, replace 'email_id' with 'expected_email_id'
unmatched_rows['email_id'] = unmatched_rows['expected_email_id']

# check sample
unmatched_rows.sample(5)

C:\Users\user\AppData\Local\Temp\ipykernel_11952\2521713817.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmatched_rows['email_id'] = unmatched_rows['expected_email_id']


,id,name,title,expected_email_id,email_id,domain,TLD,country,age,matched
4077819,3669754,Lisa Ortiz,None,lisaortiz,lisaortiz,@example,.com,Germany,61,False
1739620,1565174,Monica Gregory,None,monicagregory,monicagregory,@example,.com,Malaysia,40,False
3820344,3437752,Tracy Rasmussen,None,tracyrasmussen,tracyrasmussen,@example,.com,China,5,False
2245051,2019740,Shannon Fernandez,None,shannonfernandez,shannonfernandez,@example,.com,New Zealand,55,False
865832,778966,Jill Delacruz,None,jilldelacruz,jilldelacruz,@example,.com,Malaysia,74,False


#### Merge the matched rows and fixed unmatched rows 

In [91]:
## Code snippet to merge data ~ ..
merged_df = pd.merge(matched_rows, unmatched_rows, how='outer')

In [92]:
# check shape of newly merged df, make sure 5 Million rows
merged_df.shape

(5000000, 10)

In [93]:
merged_df.sample(5)

,id,name,title,expected_email_id,email_id,domain,TLD,country,age,matched
1004547,903648,Linda Sloan,None,lindasloan,lindasloan,@example,.com,Malaysia,59,False
4272690,3845129,Mark Carney,None,markcarney,markcarney,@example,.com,Singapore,39,False
2353448,2117274,Jordan Cole,None,jordancole,jordancole,@example,.com,Malaysia,67,False
2837448,2553166,Janet Beck,None,janetbeck,janetbeck,@example,.com,Germany,12,False
773100,695495,Alicia Wade,None,aliciawade,aliciawade,@example,.com,Malaysia,6,False


#### Drop 'expected_email_id' and 'matched' columns

In [94]:
# Drop columns
new_merged_df = merged_df.drop(['expected_email_id', 'matched'], axis=1)

In [95]:
# check sample after drop table
new_merged_df.sample(5)

,id,name,title,email_id,domain,TLD,country,age
3597778,3237321,Franklin Parker,None,franklinparker,@example,.com,USA,30
3693363,3323407,William Williams,None,williamwilliams,@example,.com,UK,76
4209296,3788031,Kathryn Stafford,None,kathrynstafford,@example,.com,Malaysia,49
128994,116145,Kristin Warner,None,kristinwarner,@example,.com,USA,40
4412830,3971314,Amanda Edwards,None,amandaedwards,@example,.com,France,69


In [96]:
# create copy of 'new_merged_df'  
new_merged_df_example = new_merged_df.copy()

In [97]:
# merge (email_id, domain, TLD) => 'email'
new_merged_df_example['email'] = new_merged_df_example['email_id'] + new_merged_df_example['domain'] + new_merged_df_example['TLD']

In [98]:
new_merged_df_example.sample(3)

,id,name,title,email_id,domain,TLD,country,age,email
3813842,3431883,Caroline Schwartz,None,carolineschwartz,@example,.com,Australia,76,carolineschwartz@example.com
3823493,3440595,Stacey Brown,None,staceybrown,@example,.com,Malaysia,77,staceybrown@example.com
134644,121204,Amy Murphy,None,amymurphy,@example,.com,Nigeria,17,amymurphy@example.com


In [99]:
# rearrange columns into 'id', 'name','title', 'email_id', 'domain', 'country', 'age'
rearrange_col_nm = ['id','name','title','email','country','age']

# Reassign to new_df_2 with the desired column
new_merged_df_example = new_merged_df_example[rearrange_col_nm]

In [100]:
print(new_merged_df_example[new_merged_df_example['id'].isin(range(1, 9))].sort_values(by='id', ascending=True))

   id              name title                        email      country  age
0   1     Gloria Montes  None     gloriamontes@example.com     Malaysia   68
1   2  Ricardo Anderson   PhD  ricardoanderson@example.com  New Zealand   11
2   3        Sara Lewis  None        saralewis@example.com      Germany   21
3   4    Troy Maldonado  None    troymaldonado@example.com    Australia   68
4   5   Cheyenne Baxter  None   cheyennebaxter@example.com        China   20
5   6       David Smith  None       davidsmith@example.com     Malaysia   22
6   6       David Smith  None       davidsmith@example.com     Malaysia   22
7   7       David Lyons  None       davidlyons@example.com           UK   78
8   8     Tommy Johnson  None     tommyjohnson@example.com           UK   31


#### Notice the name and email is accurate, however there are problems with duplication
- continue working with variable 'new_merged_df'

In [101]:
new_merged_df.sample(3)

,id,name,title,email_id,domain,TLD,country,age
2830891,2547252,Anne Smith,None,annesmith,@example,.com,UK,48
3436904,3092522,Jacob Hoffman,None,jacobhoffman,@example,.com,Nigeria,55
3905811,3514732,Jeremy Gates,None,jeremygates,@example,.com,India,22


## id is added into the 'email_id' and 'domain' to mitigate the duplication issues and achieve uniqueness as well

In [102]:
# Merge 'email_id', 'id' and 'domain' columns into a new column 'email' (added ID for uniqueness)
new_merged_df['email'] = new_merged_df['email_id'] + '_' + new_merged_df['id'].astype(str)+ new_merged_df['domain'] + new_merged_df['TLD']

In [103]:
new_merged_df.sample(3)

,id,name,title,email_id,domain,TLD,country,age,email
1562237,1405579,Michael Nicholson,None,michaelnicholson,@example,.com,Singapore,43,michaelnicholson_1405579@example.com
2961277,2664558,William Hayes,None,williamhayes,@example,.com,India,9,williamhayes_2664558@example.com
82441,74203,Jennifer Smith,None,jennifersmith,@example,.com,Malaysia,22,jennifersmith_74203@example.com


In [104]:
# Drop the original 'email_id' and 'domain' columns
new_merged_df = new_merged_df.drop(['email_id', 'domain', 'TLD'], axis=1)

In [105]:
new_merged_df.sample(3)

,id,name,title,country,age,email
3019221,2716670,Victor Duke,None,China,35,victorduke_2716670@example.com
4520801,4068530,Lynn Abbott,None,Australia,10,lynnabbott_4068530@example.com
4797826,4318005,Andrew Gomez,None,Malaysia,63,andrewgomez_4318005@example.com


In [106]:
# state the desired column 
desired_cols_acc = ['id','name','title','email','country','age']

new_merged_df = new_merged_df[desired_cols_acc]

In [107]:
new_merged_df.sample(3)

,id,name,title,email,country,age
767310,690264,Kathy Flores,None,kathyflores_690264@example.com,Germany,30
4888461,4399572,Julian Pena,None,julianpena_4399572@example.com,Singapore,64
3465335,3118077,Janice Mcfarland,MD,janicemcfarland_3118077@example.com,UK,39


In [108]:
print(new_merged_df[new_merged_df['id'].isin(range(1, 9))].sort_values(by='id', ascending=True))

   id              name title                          email      country  age
0   1     Gloria Montes  None     gloriamontes_1@example.com     Malaysia   68
1   2  Ricardo Anderson   PhD  ricardoanderson_2@example.com  New Zealand   11
2   3        Sara Lewis  None        saralewis_3@example.com      Germany   21
3   4    Troy Maldonado  None    troymaldonado_4@example.com    Australia   68
4   5   Cheyenne Baxter  None   cheyennebaxter_5@example.com        China   20
5   6       David Smith  None       davidsmith_6@example.com     Malaysia   22
6   6       David Smith  None       davidsmith_6@example.com     Malaysia   22
7   7       David Lyons  None       davidlyons_7@example.com           UK   78
8   8     Tommy Johnson  None     tommyjohnson_8@example.com           UK   31


* Continue next process

## Validity Process - Scenario 1: Validity based on title and age
- Notices that some of the title are still exists at 1 years old whih is not valid.
- Feedback shows that average age to have doctor title
    - DDS, DVM and PhD : 26 Years old
    - MD : 33 years old, due to additional 3-8 years of residency
- Therefore,
   - Rejected dataset - Those who are younger than 26 years old for DDS, DVM and PhD & younger than 33 years old for MD
   - Accepted dataset - Those who do not meet the requirements

In [109]:
# Check if 'title' column contains 'Jr.'
search_string = 'DVM'
mask = new_merged_df['title'].astype(str).str.contains(search_string, case=False)

# Filter DataFrame based on the mask
result = new_merged_df[mask]

print(result)


              id                name title  \
10            10     Benjamin Joseph   DVM   
33            32           Mike Cook   DVM   
373          343        Sherry Avery   DVM   
412          379        Janet Dennis   DVM   
413          379        Janet Dennis   DVM   
...          ...                 ...   ...   
4999045  4499138  Catherine Gonzalez   DVM   
4999046  4499138  Catherine Gonzalez   DVM   
4999129  4499214      Bridget Norman   DVM   
4999437  4499496         James Brown   DVM   
4999740  4499769        Diane Wilson   DVM   

                                         email      country  age  
10               benjaminjoseph_10@example.com      Germany   98  
33                     mikecook_32@example.com  New Zealand   67  
373                sherryavery_343@example.com           UK   61  
412                janetdennis_379@example.com           UK    5  
413                janetdennis_379@example.com           UK    5  
...                                        ..

In [110]:
# copy df_new for backup
new_merged_df_copy = new_merged_df.copy()

### Dividing accepted dataset and rejected dataset
   
##### Rejected dataset
The dataset contains of invalid data which;
1. Younger than 26 years old for DDS, DVM and PhD 
2. Younger than 33 years old for MD


##### Accepted Dataset
The dataset contains of valid data which;
1. Those who do not meet the requirement

In [111]:
# Create masks for different titles
has_DDS_DVM_PhD = new_merged_df_copy['title'].str.contains(r'(DDS|DVM|PhD)', na=False)
has_MD = new_merged_df_copy['title'].str.contains(r'MD', na=False)

# Define rejected condition based on age and title
rejected_condition = (
    ((new_merged_df_copy['age'] < 26) & (has_DDS_DVM_PhD)) |                   ## df_extension_copy
    (( new_merged_df_copy['age'] < 33) & (has_MD))
)

# Filter the DataFrame for rejected rows based on the rejected condition
rejected_filter = new_merged_df_copy[rejected_condition]

# Define accepted condition as the negation of the rejected condition
acceptable_condition = ~rejected_condition

# Filter the DataFrame based on the acceptable condition
accept_filter =  new_merged_df_copy[acceptable_condition]

C:\Users\user\AppData\Local\Temp\ipykernel_11952\880087165.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  has_DDS_DVM_PhD = new_merged_df_copy['title'].str.contains(r'(DDS|DVM|PhD)', na=False)


In [112]:
# Print filtered DataFrame
print("Accepted DataFrame:")
accept_filter.sample(5)

Accepted DataFrame:


,id,name,title,email,country,age
966818,869689,David Rogers,None,davidrogers_869689@example.com,Nigeria,70
3737217,3362932,Christian Levy,None,christianlevy_3362932@example.com,Malaysia,95
749025,673828,Linda Vaughn,None,lindavaughn_673828@example.com,Australia,21
141044,126941,Steven Rodriguez,None,stevenrodriguez_126941@example.com,Singapore,62
2866317,2579102,Hayley Lawrence,None,hayleylawrence_2579102@example.com,Malaysia,32


In [113]:
print("Rejected DataFrame:")
rejected_filter.sample(15)

Rejected DataFrame:


,id,name,title,email,country,age
1998424,1797894,Gregory Riley,MD,gregoryriley_1797894@example.com,France,19
3143360,2828524,Melissa Smith,MD,melissasmith_2828524@example.com,France,5
401210,360901,Elizabeth Mckee,MD,elizabethmckee_360901@example.com,Singapore,19
4446692,4001751,Jennifer Stewart,MD,jenniferstewart_4001751@example.com,Germany,20
784894,706128,Jack Zimmerman,DVM,jackzimmerman_706128@example.com,Nigeria,19
1672145,1504505,Yolanda Booker,MD,yolandabooker_1504505@example.com,China,7
1910291,1718527,Kristi Garcia,DDS,kristigarcia_1718527@example.com,France,9
4240874,3816480,Robert Adams,MD,robertadams_3816480@example.com,Australia,10
722464,649985,Sheri Bryant,MD,sheribryant_649985@example.com,UK,18
1285333,1156377,Heather Mueller,MD,heathermueller_1156377@example.com,Singapore,4


In [114]:
accept_filter.shape

(4963460, 6)

In [115]:
rejected_filter.shape

(36540, 6)

In [116]:
accept_filter.sample(3)

,id,name,title,email,country,age
4402533,3962000,Paula Williamson,None,paulawilliamson_3962000@example.com,China,76
3744644,3369632,Benjamin Marsh,None,benjaminmarsh_3369632@example.com,Australia,99
3619848,3257200,Adrian Henson,None,adrianhenson_3257200@example.com,France,90


In [117]:
# Check the minimum value in the 'title' column
min_age = rejected_filter['age'].max()

print("Minimum age in the 'title' column:", min_age)

Minimum age in the 'title' column: 32


#### Validate the Accepted dataset using  'search_string' with accepted row

In [118]:
# Check if 'title' column contains 'Dustin Miller.'
search_string = 'Dustin Miller'
mask = accept_filter['name'].astype(str).str.contains(search_string, case=False)

# Filter DataFrame based on the mask
result = accept_filter[mask]

In [119]:
result.sample(20)

,id,name,title,email,country,age
2281411,2052466,Dustin Miller,None,dustinmiller_2052466@example.com,New Zealand,82
3555765,3199523,Dustin Miller,None,dustinmiller_3199523@example.com,New Zealand,22
2970693,2673048,Dustin Miller,None,dustinmiller_2673048@example.com,Canada,35
1920489,1727738,Dustin Miller,None,dustinmiller_1727738@example.com,Malaysia,48
1325589,1192669,Dustin Miller,None,dustinmiller_1192669@example.com,India,2
240501,216414,Dustin Miller,None,dustinmiller_216414@example.com,India,70
194399,174968,Dustin Miller,DDS,dustinmiller_174968@example.com,Nigeria,33
3831902,3448157,Dustin Miller,None,dustinmiller_3448157@example.com,Canada,56
2354833,2118512,Dustin Miller,None,dustinmiller_2118512@example.com,New Zealand,39
2526995,2273677,Dustin Miller,None,dustinmiller_2273677@example.com,New Zealand,99


#### Modify the rejected dataset
- Replace all rejected value into "N/A" 


In [120]:
rejected_filter.shape

(36540, 6)

In [121]:
# Create a condition to identify rows for replacement
condition = (rejected_filter['title'].notnull())

# Replace values in 'title' column with "N/A" based on the condition
rejected_filter.loc[condition, 'title'] = 'N/A'

In [122]:
rejected_filter.sample(5)

,id,name,title,email,country,age
670845,603495,Ian Nelson,N/A,iannelson_603495@example.com,Germany,26
2545092,2289949,Cristina Ward,N/A,cristinaward_2289949@example.com,Malaysia,3
3346707,3011560,Ashley Ingram,N/A,ashleyingram_3011560@example.com,Malaysia,10
3613167,3251157,Amy Dixon,N/A,amydixon_3251157@example.com,UK,8
4010487,3609092,Jonathan Brown,N/A,jonathanbrown_3609092@example.com,France,23


#### Merged modified rejected with accepted value

In [123]:
import pandas as pd
## Code snippet to merge data ~ .. 'outer' is like full join
dqValidity1 = pd.merge(rejected_filter,accept_filter,how='outer')

In [124]:
dqValidity1.sample(10)

,id,name,title,email,country,age
3882409,3493645,Christian Martinez,None,christianmartinez_3493645@example.com,Malaysia,63
4337958,3903932,Sarah Reilly,None,sarahreilly_3903932@example.com,France,77
324723,292082,Lisa Sheppard,None,lisasheppard_292082@example.com,France,58
32171,28994,Nichole Evans,None,nicholeevans_28994@example.com,China,36
4305676,3874905,Nicholas Johnson,None,nicholasjohnson_3874905@example.com,Canada,3
1026066,922948,Renee Cook,None,reneecook_922948@example.com,France,55
3087348,2778005,Steven Maddox,None,stevenmaddox_2778005@example.com,Australia,53
2504361,2253276,Paul Fry,None,paulfry_2253276@example.com,Malaysia,2
2334234,2099986,Laura Shields,None,laurashields_2099986@example.com,Malaysia,5
1693606,1523855,Emily Durham,None,emilydurham_1523855@example.com,Malaysia,58


## Validity Process - Scenario 2: Validity based on email and age
   - Rejected dataset - Those who are younger than 13 years old 
   - Accepted dataset - Those who do not meet the requirements

### Dividing accepted dataset and rejected dataset
   
##### Rejected dataset
The dataset contains of invalid data which;
1. Those who are younger than 13 years old


##### Accepted Dataset
The dataset contains of valid data which;
1. Those who do not meet the requirement

In [125]:
# Define rejected & accepted condition based on age and title
rejected_condition = dqValidity1['age'] <= 12
accepted_condition = dqValidity1['age'] > 12

# Filter the DataFrame for rejected rows based on the rejected condition
rejected_filter = dqValidity1[rejected_condition]

# Filter the DataFrame based on the acceptable condition
accept_filter = dqValidity1[accepted_condition]

In [126]:
print("Rejected DataFrame:")
rejected_filter.sample(10)

Rejected DataFrame:


,id,name,title,email,country,age
3696988,3326665,Judith Walker,None,judithwalker_3326665@example.com,China,8
1165393,1048326,Jacob Kerr,None,jacobkerr_1048326@example.com,Canada,1
2764346,2487385,Brent Lopez,None,brentlopez_2487385@example.com,India,7
1880950,1692151,Samantha Bell,None,samanthabell_1692151@example.com,New Zealand,6
4305912,3875118,Haley Ramirez,None,haleyramirez_3875118@example.com,UK,8
252928,227622,Katherine Snow,None,katherinesnow_227622@example.com,India,1
4893515,4404093,Sarah Johnson,None,sarahjohnson_4404093@example.com,Australia,9
855901,770027,Cory Pollard,None,corypollard_770027@example.com,Malaysia,12
3461152,3114341,Shannon Wilson,N/A,shannonwilson_3114341@example.com,Nigeria,3
3900832,3510250,Valerie Banks,None,valeriebanks_3510250@example.com,France,12


In [127]:
print("Accepted DataFrame:")
accept_filter.sample(10)

Accepted DataFrame:


,id,name,title,email,country,age
2941485,2646768,Michelle Burke,None,michelleburke_2646768@example.com,India,52
3537242,3182887,Yvonne Johnson,None,yvonnejohnson_3182887@example.com,UK,51
448936,403927,Debbie Perez,MD,debbieperez_403927@example.com,Singapore,43
4387320,3948305,Daniel Wilson,None,danielwilson_3948305@example.com,UK,46
1711119,1539545,Elizabeth Bryant,None,elizabethbryant_1539545@example.com,USA,72
4255750,3829850,Arthur Adams,None,arthuradams_3829850@example.com,India,94
1030060,926537,Deanna Bennett,None,deannabennett_926537@example.com,New Zealand,82
4295334,3865588,Monica Montgomery,None,monicamontgomery_3865588@example.com,Australia,46
1216027,1094058,Jason Miller,None,jasonmiller_1094058@example.com,Singapore,98
3107973,2796591,Laura Brown,None,laurabrown_2796591@example.com,China,77


### Modified Rejected dataset
- Replace email value with "N/A" for those younger than 12 years old

In [128]:
# Create a condition to identify rows for replacement
# condition = rejected_filter[(rejected_filter['age'] <= 12) & (rejected_filter['email'].str.contains('@'))]
condition = (rejected_filter['age'] <= 12)

# Replace values in 'title' column with "N/A" based on the condition
rejected_filter.loc[condition, 'email'] = 'N/A'

In [129]:
print("Updated DataFrame:")
rejected_filter.sample(10)

Updated DataFrame:


,id,name,title,email,country,age
4819469,4337506,Jessica Hunter,None,N/A,India,2
4978853,4480976,Benjamin Banks,N/A,N/A,Nigeria,4
1505097,1354129,Elizabeth Walker,None,N/A,Malaysia,9
4513381,4061857,Karen Horn,None,N/A,Malaysia,10
474572,427025,Benjamin Barnett,None,N/A,New Zealand,3
3305913,2974792,Collin Romero,None,N/A,USA,3
3618833,3256272,Michelle Lawson,None,N/A,Malaysia,3
1443815,1298977,Aaron Rojas,None,N/A,UK,10
4984804,4486336,Jason Torres,None,N/A,Nigeria,4
881089,792727,Lisa Wright,None,N/A,New Zealand,8


In [130]:
rejected_filter.shape

(598780, 6)

In [131]:
accept_filter.shape

(4401220, 6)

#### Merging the modified dataset with accepted dataset

In [132]:
import pandas as pd
## Code snippet to merge data ~ .. 'outer' is like full join
dqValidity2 = pd.merge(rejected_filter,accept_filter,how='outer')

In [133]:
dqValidity2.shape

(5000000, 6)

In [134]:
dqValidity2.sample(10)

,id,name,title,email,country,age
4228521,3805349,Lindsay Reed,None,lindsayreed_3805349@example.com,Germany,87
2815696,2533575,Robin Webb,None,robinwebb_2533575@example.com,Malaysia,85
3020380,2717710,Tonya Keith,None,tonyakeith_2717710@example.com,Singapore,36
1139979,1025415,Alexandra Moreno,None,alexandramoreno_1025415@example.com,China,14
4026147,3623129,Nicole Joyce,N/A,nicolejoyce_3623129@example.com,Germany,25
272660,245308,Melissa Smith,None,melissasmith_245308@example.com,Germany,54
4824861,4342340,Cheryl Brooks,MD,cherylbrooks_4342340@example.com,Nigeria,41
887187,798187,Katherine Johnson,None,katherinejohnson_798187@example.com,Malaysia,50
2073510,1865439,Edwin Nicholson,None,edwinnicholson_1865439@example.com,Australia,95
3467245,3119826,Shannon Clark,None,shannonclark_3119826@example.com,Australia,80


## Completeness - Remove missing value in title column
- Checking if any column consists of null value

In [135]:
dqValidity2.isnull().values.any()

True

In [136]:
null_counts = dqValidity2.isnull().sum()
print(null_counts)

id               0
name             0
title      4853727
email            0
country          0
age              0
dtype: int64


#### Dividing the Accepted dataset
- not null value

In [137]:
# Filter rows where 'title' column is not null
not_null_rows = dqValidity2[dqValidity2['title'].notnull()]

# Print the filtered DataFrame
print("Rows where 'title' column is not null:")
not_null_rows.sample(10)

Rows where 'title' column is not null:


,id,name,title,email,country,age
4715494,4244042,Amanda Wilson,MD,amandawilson_4244042@example.com,France,46
111461,100346,Donna Atkinson,PhD,donnaatkinson_100346@example.com,Malaysia,77
1405301,1264361,Christine Leblanc,N/A,christineleblanc_1264361@example.com,Malaysia,19
1208979,1087678,Rhonda Sutton,N/A,rhondasutton_1087678@example.com,Nigeria,26
688430,619308,Gregory Rodriguez,DVM,gregoryrodriguez_619308@example.com,New Zealand,50
3169091,2851645,Gerald Williams,Jr.,geraldwilliams_2851645@example.com,Canada,16
4717461,4245802,Jessica Santiago,PhD,jessicasantiago_4245802@example.com,UK,90
1517808,1365575,Kevin Johnson,PhD,kevinjohnson_1365575@example.com,Nigeria,51
4000834,3600354,Janice Case,MD,janicecase_3600354@example.com,France,77
3605547,3244306,Joshua Ellis,MD,joshuaellis_3244306@example.com,Canada,83


In [138]:
not_null_rows.shape

(146273, 6)

#### Dividing the Rejected dataset
- Null value

In [139]:
null_rows = dqValidity2[dqValidity2['title'].isnull()]

In [140]:
null_rows.shape

(4853727, 6)

In [141]:
null_rows.sample(10)

,id,name,title,email,country,age
4171640,3754138,Colton Roth,None,coltonroth_3754138@example.com,Singapore,76
924486,831665,Franklin Erickson,None,franklinerickson_831665@example.com,Malaysia,45
4366083,3929182,Karen Smith,None,karensmith_3929182@example.com,Germany,32
365659,328874,Kimberly Green,None,kimberlygreen_328874@example.com,New Zealand,72
1329089,1195819,John Robinson,None,johnrobinson_1195819@example.com,Australia,51
2335459,2101089,Pamela Robinson,None,pamelarobinson_2101089@example.com,Australia,24
599087,538887,Makayla Robinson,None,N/A,Canada,11
3989983,3590574,Craig Arnold,None,N/A,Singapore,10
2613824,2351859,Karen Hernandez,None,karenhernandez_2351859@example.com,France,58
2726370,2453134,Robert Sloan,None,robertsloan_2453134@example.com,USA,24


### Modifying the Rejected dataset

In [142]:
null_rows.shape

(4853727, 6)

In [143]:
# Fill missing values in 'title' column with 'Not Applicable' and assign to a variable
# modify_null_rows = null_rows.copy()

# Replace null values in 'title' column with 'Not Applicable'
null_rows.loc[null_rows['title'].isnull(), 'title'] = 'N/A'


In [144]:
# after replace null to N/A
null_rows.shape

(4853727, 6)

In [145]:
# Print updated DataFrame
print("DataFrame with Nulls Replaced:")
null_rows.sample(5)

DataFrame with Nulls Replaced:


,id,name,title,email,country,age
4322629,3890177,Jerry Cross,N/A,jerrycross_3890177@example.com,Nigeria,72
4256118,3830178,Scott House,N/A,N/A,China,7
1862311,1675373,Cody Foley,N/A,codyfoley_1675373@example.com,Australia,44
2880186,2591611,Cindy Williams,N/A,cindywilliams_2591611@example.com,UK,64
3876215,3488083,Stacey Torres,N/A,staceytorres_3488083@example.com,Malaysia,47


#### Merge the modified rejected dataset with accepted dataset

In [146]:
import pandas as pd
## Code snippet to merge data ~ .. 'outer' is like full join
dqCompleteness = pd.merge(null_rows,not_null_rows,how='outer')

In [147]:
dqCompleteness.shape

(5000000, 6)

In [148]:
null_count = dqCompleteness.isnull().sum()
print(null_count)

id         0
name       0
title      0
email      0
country    0
age        0
dtype: int64


### Optional to save as parquet file

In [149]:
# Save the DataFrame to a Parquet file
dqCompleteness.to_parquet('dqCompletenessBeforeUniqueness.parquet', index=False)
print("Data saved to dqCompletenessBeforeUniqueness.parquet file.")

Data saved to dqCompletenessBeforeUniqueness.parquet file.


# 5. Uniqueness Process
- Check the duplicate value based on id and name column

In [150]:
dp_copy_1 = dqCompleteness.copy()


In [151]:
# Check any column with the string 
search_string = 'Michael Watson'
mask1 = dp_copy_1.apply(lambda x: x.astype(str).str.contains(search_string, case=False)).any(axis=1)
result1 = dp_copy_1[mask1]
result1.sample(10)

,id,name,title,email,country,age
3125418,2812303,Michael Watson,N/A,michaelwatson_2812303@example.com,New Zealand,53
809840,728528,Michael Watson,N/A,michaelwatson_728528@example.com,USA,97
3658358,3291814,Michael Watson,N/A,michaelwatson_3291814@example.com,Nigeria,32
3617681,3255236,Michael Watson,DDS,michaelwatson_3255236@example.com,Australia,66
2045007,1839839,Michael Watson,N/A,michaelwatson_1839839@example.com,USA,93
2564117,2307112,Michael Watson,N/A,michaelwatson_2307112@example.com,UK,32
4590022,4130930,Michael Watson,N/A,N/A,France,9
3509543,3157965,Michael Watson,N/A,michaelwatson_3157965@example.com,New Zealand,82
1025398,922351,Michael Watson,N/A,michaelwatson_922351@example.com,Singapore,56
1198114,1077863,Michael Watson,N/A,michaelwatson_1077863@example.com,UK,57


#### Dividing the Accepted and Rejected Dataset
1. Accepted  dataset - Not duplicate data
2. Rejected dataset - Duplicate data

- 'keep=False' means that all instances of a duplicated row are marked as duplicates. 
- This includes both the first occurrence of the duplicated row and all subsequent occurrences.
- To easily identify and work with all duplicate entries in DataFrame.

In [152]:
# duplicate_mask = dp_copy.duplicated(subset=['id','name','title'], keep=False)
duplicate_mask = dp_copy_1.duplicated(subset=['id','name'], keep=False)

#Duplicate data
duplicate_ID_Name = dp_copy_1[duplicate_mask]

#non duplicate data
non_duplicate_ID_Name = dp_copy_1[~duplicate_mask]

In [153]:
#Duplicate value
print("\nDuplicate Data:")
duplicate_ID_Name.sample(10)


Duplicate Data:


,id,name,title,email,country,age
375467,337715,Jesse Ramirez,N/A,jesseramirez_337715@example.com,Singapore,96
2627207,2363897,Cheryl Benson,N/A,cherylbenson_2363897@example.com,Singapore,73
4574136,4116658,Roger Rivera,N/A,rogerrivera_4116658@example.com,New Zealand,49
933515,839767,Alexander Soto,N/A,alexandersoto_839767@example.com,China,64
4661532,4195322,Steven Johnson,N/A,stevenjohnson_4195322@example.com,India,60
4514423,4062775,Richard Kent,N/A,richardkent_4062775@example.com,Malaysia,93
3134816,2820813,Joseph Doyle,N/A,josephdoyle_2820813@example.com,Australia,74
331988,298614,Heather Reed,N/A,heatherreed_298614@example.com,Singapore,96
4400518,3960200,Virginia Herrera,N/A,N/A,USA,7
4073662,3666005,Gregory Thompson,N/A,gregorythompson_3666005@example.com,New Zealand,39


In [154]:
print("\nNot Duplicate Data:")
non_duplicate_ID_Name.sample(10)


Not Duplicate Data:


,id,name,title,email,country,age
4212124,3790582,Natasha Chen,N/A,N/A,Nigeria,6
31268,28198,Roger Henry,N/A,rogerhenry_28198@example.com,Canada,18
875106,787341,James Pena,N/A,N/A,France,5
2730293,2456692,Randy Moore,N/A,randymoore_2456692@example.com,Malaysia,13
1071498,963802,Ruth Jenkins,N/A,ruthjenkins_963802@example.com,Germany,86
3159570,2843080,Anna Sellers,N/A,annasellers_2843080@example.com,China,73
2480827,2232059,Allison Dougherty,N/A,allisondougherty_2232059@example.com,New Zealand,77
3835517,3451413,Deborah Gonzalez,N/A,deborahgonzalez_3451413@example.com,Germany,48
677845,609785,Joseph Franklin,N/A,josephfranklin_609785@example.com,Malaysia,52
3207431,2886065,Karen Trujillo,N/A,karentrujillo_2886065@example.com,China,56


#### Modify the rejected dataset - Uniqueness Process
1. Remove the duplicate value based on id and name

In [155]:
deduplicate_ID_Name_df = duplicate_ID_Name.drop_duplicates(subset=['id','name'])

# Display the deduplicated DataFrame
print("\nDeduplicated DataFrame based on ID and Name:")
deduplicate_ID_Name_df.sample(10)


Deduplicated DataFrame based on ID and Name:


,id,name,title,email,country,age
190471,171416,Eric James,N/A,ericjames_171416@example.com,India,44
4848062,4363176,Kelsey Watson,N/A,kelseywatson_4363176@example.com,New Zealand,27
915021,823155,Kimberly Reed,N/A,N/A,France,3
3612909,3250929,Debra Wilkinson,N/A,debrawilkinson_3250929@example.com,Singapore,69
1261741,1135223,Austin Wilkinson,N/A,austinwilkinson_1135223@example.com,UK,16
4464302,4017623,Robert Lewis,N/A,N/A,Malaysia,2
1086493,977337,Christina Santiago,N/A,christinasantiago_977337@example.com,Australia,35
3108019,2796631,Julian Yu,N/A,julianyu_2796631@example.com,New Zealand,96
4590539,4131397,Michelle Kennedy,N/A,michellekennedy_4131397@example.com,Nigeria,81
1755828,1579769,Jenna Rodriguez,N/A,jennarodriguez_1579769@example.com,USA,81


#### Merge rejected dataset and accepted dataset

In [156]:
import pandas as pd
## Code snippet to merge data ~ .. 'outer' is like full join
dqUniquenessIDName = pd.merge(deduplicate_ID_Name_df,non_duplicate_ID_Name,how='outer')

In [157]:
dqUniquenessIDName.shape

(4500000, 6)

In [158]:
dqUniquenessIDName.sample(5)

,id,name,title,email,country,age
1391762,1391763,Lindsay Miller,N/A,lindsaymiller_1391763@example.com,UK,25
455388,455389,Rebecca Lopez,N/A,rebeccalopez_455389@example.com,Singapore,44
1423927,1423928,Nicole Howard,N/A,nicolehoward_1423928@example.com,Singapore,58
3572282,3572283,Julie Pearson,N/A,juliepearson_3572283@example.com,Malaysia,73
3547997,3547998,Brian Mccarty,N/A,brianmccarty_3547998@example.com,Malaysia,53


### Optional to save as parquet

In [159]:
# Save the DataFrame to a Parquet file
dqUniquenessIDName.to_parquet('dqdatademo_processed.parquet', index=False)
print("Data saved to dqdatademo_processed.parquet file.")

Data saved to dqdatademo_processed.parquet file.


#### Ensure there are no duplicate value

In [160]:
# Check any column with the string 
search_string = 'Michael Watson'
mask = dqUniquenessIDName.apply(lambda x: x.astype(str).str.contains(search_string, case=False)).any(axis=1)
result = dqUniquenessIDName[mask]
print(result)

              id            name title                              email  \
10848      10849  Michael Watson   N/A    michaelwatson_10849@example.com   
27644      27645  Michael Watson   N/A    michaelwatson_27645@example.com   
44742      44743  Michael Watson   N/A    michaelwatson_44743@example.com   
120251    120252  Michael Watson   N/A   michaelwatson_120252@example.com   
122351    122352  Michael Watson   N/A   michaelwatson_122352@example.com   
...          ...             ...   ...                                ...   
4383145  4383146  Michael Watson   N/A  michaelwatson_4383146@example.com   
4444522  4444523  Michael Watson   N/A                                N/A   
4454429  4454430  Michael Watson   N/A  michaelwatson_4454430@example.com   
4467075  4467076  Michael Watson   N/A  michaelwatson_4467076@example.com   
4473830  4473831  Michael Watson   N/A  michaelwatson_4473831@example.com   

             country  age  
10848          India   85  
27644    New Zealan

In [161]:
# Step 2: Check for duplicates within the identified rows
duplicates = result[result.duplicated()]

# print("Rows containing the search string 'Cody Ross':")
# print(result)
print("\nDuplicate rows containing the search string 'Michael Watson':")
print(duplicates)


Duplicate rows containing the search string 'Michael Watson':
Empty DataFrame
Columns: [id, name, title, email, country, age]
Index: []


# To upload into Azure Blob

In [162]:
# Convert DataFrame to Parquet format
parquet_file_path = "dqdatademo_processed.parquet"
table = pa.Table.from_pandas(dqUniquenessIDName)
pq.write_table(table, parquet_file_path)

In [163]:
print(parquet_file_path)

dqdatademo_processed.parquet


In [164]:
import pandas as pd

# Read the Parquet file into a DataFrame
df_from_parquet = pd.read_parquet(parquet_file_path)

In [165]:
df_from_parquet.sample(5)

,id,name,title,email,country,age
1879754,1879755,Michael Smith,N/A,michaelsmith_1879755@example.com,UK,41
1370231,1370232,Michael Martin,N/A,michaelmartin_1370232@example.com,Nigeria,63
1733529,1733530,Nancy Burke,N/A,N/A,Nigeria,11
547399,547400,Katherine Gallagher,N/A,katherinegallagher_547400@example.com,USA,95
644135,644136,Scott Smith,N/A,scottsmith_644136@example.com,India,82


In [166]:
# Upload the Parquet file back to Azure Blob Storage
parquet_blob_client = container_client.get_blob_client("03-Silver/dqdatademo_processed.parquet")
with open(parquet_file_path, "rb") as data:
    parquet_blob_client.upload_blob(data)